In [1]:
## K-means clustering implementation. 
## Check https://www.youtube.com/watch?v=_S5tvagaQRU for more details
import random

In [2]:
# Define the number of clusters
k = 3

In [3]:
# List pairs to cluster. Each pair consists of two coordinates (x, y)
coordinates = [
    (2, 10),
    (2, 5),
    (8, 4),
    (5, 8),
    (7, 5),
    (6, 4),
    (1, 2),
    (4, 9)
]
coordinates

[(2, 10), (2, 5), (8, 4), (5, 8), (7, 5), (6, 4), (1, 2), (4, 9)]

In [4]:
# Randomly select the initial centroids
centroids = random.sample(coordinates, k)
centroids = [(5, 8),(5, 8),(5, 8)]
centroids

[(5, 8), (5, 8), (5, 8)]

In [5]:
# Calculates the distance of pair to centroids and returns the closest centroid to that pair
def assign_to_cluster(pair, centroids): 
    distance_final = 9999999999
    cluster = 9999
    for each_centroid in centroids:
        distance = abs(each_centroid[0] - pair[0]) + abs(each_centroid[1] - pair[1])
        if distance < distance_final:
            cluster = centroids.index(each_centroid)
            distance_final = distance
        #print(each_centroid, pair, distance, cluster, distance_final)
    
    return pair[0], pair[1], cluster

In [6]:
# Cluster the list of pair to closest centroid
def generate_clusters(coordinates, centroids):
    li = []
    for each_coordinate in coordinates:
        li.append(assign_to_cluster(each_coordinate, centroids))

    coordinates = li

    li = []
    for each_centroid in centroids:
        index = centroids.index(each_centroid)
        count = 0
        sum_x = 0
        sum_y = 0
        for each_coordinate in coordinates:
            if each_coordinate[2] == index:
                count = count + 1
                sum_x = sum_x + each_coordinate[0]
                sum_y = sum_y + each_coordinate[1]
                #print(each_centroid, each_coordinate)
        li.append((sum_x / count, sum_y / count))
    centroids = li
        
    return coordinates, centroids

In [7]:
# Iterates clustering until the best "accomodation" is found
current_centroids = []
while current_centroids != centroids:
    current_centroids = centroids
    coordinates, centroids = generate_clusters(coordinates, centroids)
    print(centroids)

[(4.375, 5.875), (4.375, 5.875), (4.375, 5.875)]
[(4.375, 5.875), (4.375, 5.875), (4.375, 5.875)]


In [8]:
coordinates

[(2, 10, 0),
 (2, 5, 0),
 (8, 4, 0),
 (5, 8, 0),
 (7, 5, 0),
 (6, 4, 0),
 (1, 2, 0),
 (4, 9, 0)]

In [9]:
"""
[(2, 10, 1),
 (2, 5, 0),
 (8, 4, 2),
 (5, 8, 1),
 (7, 5, 2),
 (6, 4, 2),
 (1, 2, 0),
 (4, 9, 1)]
"""

'\n[(2, 10, 1),\n (2, 5, 0),\n (8, 4, 2),\n (5, 8, 1),\n (7, 5, 2),\n (6, 4, 2),\n (1, 2, 0),\n (4, 9, 1)]\n'

In [ ]:
def calculate_silhouette(data_points):
    intra = []
    for i, each_point in data_points.iterrows():
        neighbors = data_points
        neighbors = neighbors.drop(i)
        neighbors = neighbors[(neighbors["cx"] == each_point["cx"]) & 
                              (neighbors["cy"] == each_point["cy"]) & 
                              (neighbors["cz"] == each_point["cz"])]
        intra_cluster = find_cluster_distance([each_point.values.tolist()], neighbors.values.tolist(), type = "avg")

        other_clusters = data_points
        other_clusters = other_clusters[~((other_clusters["cx"] == each_point["cx"]) & 
                                          (other_clusters["cy"] == each_point["cy"]) & 
                                          (other_clusters["cz"] == each_point["cz"]))]    
        other_clusters = (other_clusters[["cx", "cy", "cz"]]).drop_duplicates()
        inter = []
        for j, each_non_neighbors in other_clusters.iterrows():
            non_neighbors = data_points
            non_neighbors = non_neighbors[(non_neighbors["cx"] == each_non_neighbors["cx"]) & 
                                          (non_neighbors["cy"] == each_non_neighbors["cy"]) & 
                                          (non_neighbors["cz"] == each_non_neighbors["cz"])]
            non_neighbors = non_neighbors[["x", "y", "z"]]
            inter_cluster = find_cluster_distance([each_point.values.tolist()], non_neighbors.values.tolist(), type = "avg")
            inter.append(inter_cluster)   

        b1 = min(inter)
        s1 = (b1 - intra_cluster) / max(intra_cluster, b1)
        intra.append({
                "index": i,
                "silhouette_coeficient": s1
            }
        )
    #     print("P", i, ": ", each_point["x"], each_point["y"], intra_cluster, min(inter), b1, s1)
    return pd.concat([data_points, pd.DataFrame(intra).set_index("index")], axis=1)

calculate_silhouette(all_clusters)

In [ ]:
test_without_outliers = remove_outliers(test, d_distance = 50, p_fraction = 0.15)

def k_means(data_points, k=10):
    current_centroids = np.zeros((k, 3)).tolist()
    centroids = np.random.randint(0, 100, size=(k, 3)).tolist()
    counter = 1

    while current_centroids != centroids:
        current_centroids = centroids
        # Assign data points to centroids
        data_points["nearest"] = data_points.apply(
            lambda row: find_closest_cluster(
                [[row.x, row.y, row.z]], 
                centroids, 
                type = "min"), 
            axis = 1)

        data_points["cluster"] = data_points["nearest"].astype(str)

        # Re-calculate the centroids
        centroids = data_points[["cluster", "x", "y", "z"]].groupby(["cluster"]).mean()
        centroids = centroids[["x", "y", "z"]].values.tolist()

        counter += 1
    print(f"Number of iterations needed to converge to {k} clusters:", counter)
    
    data_points["cluster"] = data_points["cluster"].str.replace("\[|\]", "", regex=True)
    data_points[["cx", "cy", "cz"]] = data_points["cluster"].str.split(",", expand=True)
    data_points[["cx", "cy", "cz"]] = data_points[["cx", "cy", "cz"]].apply(pd.to_numeric)
    return data_points.drop(["nearest", "cluster"], axis=1)

k_means(test_without_outliers, 10)

53	67	43	65.0	58.0	37.0	16.155494

In [2]:
(53 + 65) / 2, (67 + 58) / 2, (43 + 37) / 2

(59.0, 62.5, 40.0)

In [ ]:
def hierarchical_agglomerative(data_points, k=10, distance_type = "min", distances = None):
    # First we define each point belonging to its own "cluster"
    data_points["cx"] = data_points["x"]
    data_points["cy"] = data_points["y"]
    data_points["cz"] = data_points["z"]

    if distances is None:
        distances = initialize_distances(data_points.copy())

    # Agglomerate while the number of clusters is greater than k
    while len((data_points[["cx", "cy", "cz"]]).drop_duplicates().index) > k:
        # Then we find the set of two "clusters" with the shortest distance, considering the distance_type
        clusters = (data_points[["cx", "cy", "cz"]]).drop_duplicates()
        for i, each_point in clusters.iterrows():
            neighbors = data_points
            neighbors = neighbors[(neighbors["cx"] == each_point["cx"]) & 
                                  (neighbors["cy"] == each_point["cy"]) & 
                                  (neighbors["cz"] == each_point["cz"])]

            inner_distance = []
            other_clusters = data_points
            other_clusters = other_clusters[~((other_clusters["cx"] == each_point["cx"]) & 
                                              (other_clusters["cy"] == each_point["cy"]) & 
                                              (other_clusters["cz"] == each_point["cz"]))]    
            other_clusters = (other_clusters[["cx", "cy", "cz"]]).drop_duplicates()    
            for j, each_non_neighbors in other_clusters.iterrows():
                distance = (
                    distances[
                        (
                            (
                                (distances["cx1"] == each_point["cx"]) &
                                (distances["cy1"] == each_point["cy"]) &
                                (distances["cz1"] == each_point["cz"]) &
                                (distances["cx2"] == each_non_neighbors["cx"]) &
                                (distances["cy2"] == each_non_neighbors["cy"]) &
                                (distances["cz2"] == each_non_neighbors["cz"])
                            ) |
                            (
                                (distances["cx1"] == each_non_neighbors["cx"]) &
                                (distances["cy1"] == each_non_neighbors["cy"]) &
                                (distances["cz1"] == each_non_neighbors["cz"]) &
                                (distances["cx2"] == each_point["cx"]) &
                                (distances["cy2"] == each_point["cy"]) &
                                (distances["cz2"] == each_point["cz"])
                            )                       
                        )   
                    ]
                )["distance"]
                if len(distance) == 0:
                    non_neighbors = data_points
                    non_neighbors = non_neighbors[(non_neighbors["cx"] == each_non_neighbors["cx"]) & 
                                                  (non_neighbors["cy"] == each_non_neighbors["cy"]) & 
                                                  (non_neighbors["cz"] == each_non_neighbors["cz"])]
                    non_neighbors = non_neighbors[["x", "y", "z"]]        
                    cluster_distance = find_cluster_distance(
                        neighbors[["x", "y", "z"]].values.tolist(), non_neighbors.values.tolist(), type = distance_type)
                
                    #Save distances to do not have to re-calculate
                    distances = distances.append({
                        "cx1": each_point["cx"],
                        "cy1": each_point["cy"],
                        "cz1": each_point["cz"],
                        "cx2": each_non_neighbors["cx"],
                        "cy2": each_non_neighbors["cy"],
                        "cz2": each_non_neighbors["cz"],
                        "distance": cluster_distance

                    }, ignore_index = True)
                else:
                    cluster_distance = distance.values[0]                 
        
        shortest_distance_index = distances.sort_values(by=["distance"]).index[0]
        
        to_merge = data_points[((data_points["cx"] == distances.at[shortest_distance_index, "cx1"]) & 
                          (data_points["cy"] == distances.at[shortest_distance_index, "cy1"]) & 
                          (data_points["cz"] == distances.at[shortest_distance_index, "cz1"])) |
                          ((data_points["cx"] == distances.at[shortest_distance_index, "cx2"]) & 
                          (data_points["cy"] == distances.at[shortest_distance_index, "cy2"]) & 
                          (data_points["cz"] == distances.at[shortest_distance_index, "cz2"]))
                         ]
        
        distances = distances[~((distances["cx1"].isin(to_merge["cx"])) & 
                          (distances["cy1"].isin(to_merge["cy"])) & 
                          (distances["cz1"].isin(to_merge["cz"]))) &
                          ((distances["cx2"].isin(to_merge["cx"])) & 
                          (distances["cy2"].isin(to_merge["cy"])) & 
                          (distances["cz2"].isin(to_merge["cz"])))
                        ]
        
        data_points.loc[to_merge.index, ["cx"]] = to_merge["x"].mean()
        data_points.loc[to_merge.index, ["cy"]] = to_merge["y"].mean()
        data_points.loc[to_merge.index, ["cz"]] = to_merge["z"].mean()
        
        print("Agglomerating to", len((data_points[["cx", "cy", "cz"]]).drop_duplicates().index), "clusters")

    return data_points

k = 2
# test_hierarchical = remove_outliers(test, d_distance = 50, p_fraction = 0.15)
clusterized_hierarchical = hierarchical_agglomerative(test_hierarchical.head(10), k, distance_type = "min")
clusterized_hierarchical